In [31]:
# This code snippet would help to access module of upper directory

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [32]:
from read_utils import load_one_param_file

In [10]:
from utils import DATA_DIR, FIRST_EXPERIMENT_CLASSES,\
    SESSION_NAME, SESSION_OBJECTS, SESSION_EVENTS, GLYPH_BOX, NORMAL, \
    START, END, LABEL

In [11]:
session_data = load_one_param_file(os.path.join( DATA_DIR, 'SlideAround', 'Session1', 'files.param'))

'NoneType' object has no attribute 'findall'


In [15]:
import numpy as np

np.cross([0,0,1], [1,0,0])

array([0, 1, 0])

In [33]:
from feature.project_table import project_markers, estimate_cube_2d
from simulator.utils import Cube2D, Transform2D
'''
Demo the movement of objects in a session from from_frame to to_frame (inclusive)
'''
def demo_session (session_data, from_frame, to_frame):
    pass